# 2 - create_table_posts_creator

Este notebook realiza a ingestão dos dados brutos de posts de criadores de conteúdo.  
O fluxo inclui:  
- leitura de arquivo JSON/JSON.GZ parametrizado com schema fixo (views, likes, título, data, tags, yt_user);  
- normalização do campo `published_at` em `published_at_ts` (timestamp) e `published_month` (date);  
- aplicação de validações de sanidade (curtidas/visualizações não negativas, remoção de duplicados, checagem de nulos);  
- escrita dos dados em tabela Delta gerenciada, com comentários de documentação em tabela e colunas.  

O resultado é a criação/atualização da tabela **default.posts_creator**.  


In [0]:
# Libraries and configuration

from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType, ArrayType
)
import logging, json, traceback

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Widgets (parameterization)

dbutils.widgets.text(
    "input_path",
    "/Volumes/workspace/default/gz_files/posts_creator.json.gz",
    "File path (.json/.json.gz)"
)
dbutils.widgets.text(
    "output_table_name",
    "default.posts_creator",
    "Target Delta table name"
)
dbutils.widgets.dropdown(
    "multiline", "false", 
    ["false", "true"], 
    "JSON multiline? (true/false)"
)

input_path        = dbutils.widgets.get("input_path").strip()
output_table_name = dbutils.widgets.get("output_table_name").strip()
use_multiline     = dbutils.widgets.get("multiline").lower() == "true"

logging.info(f"Arquivo de origem: {input_path}")
logging.info(f"Tabela destino: {output_table_name}")
logging.info(f"Leitura multiline: {use_multiline}")


# Data contract

schema = StructType([
    StructField("creator_id",  StringType(), True),
    StructField("views",       LongType(),   True),
    StructField("likes",       LongType(),   True),
    StructField("title",       StringType(), True),
    StructField("published_at",StringType(), True),  # will be normalized
    StructField("tags",        ArrayType(StringType()), True),
    StructField("yt_user",     StringType(), True),
])


# Path validation

try:
    _ = dbutils.fs.head(input_path, 1024)
except Exception as e:
    msg = f"ERRO: caminho inválido ou inacessível: {input_path} | {e}"
    logging.error(msg)
    dbutils.notebook.exit(msg)


# JSON reading

try:
    reader = spark.read.schema(schema)
    if use_multiline:
        reader = reader.option("multiLine", "true")
    df_raw = reader.json(input_path)
except Exception as e:
    logging.error("Falha ao ler JSON:\n" + traceback.format_exc())
    dbutils.notebook.exit(f"ERRO ao ler JSON: {e}")


# Normalization / Cleaning

df = (
    df_raw
      .withColumn("creator_id", F.trim("creator_id"))
      .withColumn("title",      F.trim("title"))
      .withColumn("yt_user",    F.lower(F.trim("yt_user")))
)

# Robust conversion of published_at:
# - if numeric: detect epoch in ms (>= 13 digits) and divide by 1000
# - if ISO-8601 (string with '-' and ':'): use to_timestamp directly
df = df.withColumn(
    "published_at_ts",
    F.when(
        F.col("published_at").rlike(r"^\d+$"),
        F.to_timestamp(F.from_unixtime(
            F.when(F.length("published_at") >= 13,
                   (F.col("published_at").cast("double")/1000.0)
            ).otherwise(F.col("published_at").cast("double"))
        ))
    ).otherwise(F.to_timestamp("published_at"))
)

# sanity filters

df = df.filter((F.col("views") >= 0) & (F.col("likes") >= 0))


# extra information (useful for monthly analysis)

df = df.withColumn("published_month", F.to_date(F.date_trunc("month", F.col("published_at_ts"))))


# basic deduplication 

df = df.dropDuplicates(["creator_id", "title", "published_at"])


# empty check without isEmpty()

if df.limit(1).count() == 0:
    dbutils.notebook.exit("ERRO: nenhum registro válido após limpeza/conversão.")


# log of null timestamps (debugging bad data)

null_ts = df.filter(F.col("published_at_ts").isNull()).count()
if null_ts > 0:
    logging.warning(f"{null_ts} registro(s) com timestamp nulo em 'published_at_ts'.")


# Write to Delta

try:
    (df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(output_table_name))

    spark.sql(f"""
      COMMENT ON TABLE {output_table_name}
      IS 'Tabela de posts dos creators (views/likes/title/published_at_ts/tags/yt_user) — origem: {input_path}'
    """)
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.creator_id        IS 'Identificador do criador (fonte)';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.views             IS 'Quantidade de visualizações';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.likes             IS 'Quantidade de curtidas';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.title             IS 'Título do vídeo/post';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.published_at      IS 'Valor bruto de data/hora do dataset de origem';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.published_at_ts   IS 'Timestamp normalizado do momento de publicação';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.published_month   IS 'Primeiro dia do mês referente à publicação (DATE)';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.tags              IS 'Lista de tags do post';")
    spark.sql(f"COMMENT ON COLUMN {output_table_name}.yt_user           IS 'Identificador do canal no YouTube';")

    spark.sql(f"""
      ALTER TABLE {output_table_name}
      SET TBLPROPERTIES (
        'pipeline.step' = 'create_table_posts_creator',
        'source.path'   = '{input_path}'
      )
    """)

    logging.info(f"Tabela '{output_table_name}' criada/sobrescrita com sucesso.")
except Exception as e:
    logging.error("Falha ao escrever Delta:\n" + traceback.format_exc())
    dbutils.notebook.exit(f"ERRO ao escrever Delta: {e}")


# Return

final_count = spark.table(output_table_name).count()
payload = {
    "status": "SUCESSO",
    "source_path": input_path,
    "target_table": output_table_name,
    "records_processed": final_count
}
logging.info(f"Processo finalizado. Registros ingeridos: {final_count}")
dbutils.notebook.exit(json.dumps(payload, ensure_ascii=False))
